In [ ]:
'''
Reference:
Example script from Keras Github Repository keras/examples/lstm_text_generation.py
https://github.com/keras-team/keras

Same configuration as the vanilla LSTM char-RNN for Donna Summer, except with two layers of LSTM of 128 units
'''

from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

#path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
#text = io.open(path, encoding='utf-8').read().lower()
import pandas as pd
lyrics_data = pd.read_csv('songdata.csv')

# Donna Summer

lyrics_data_donna = lyrics_data[lyrics_data.artist == 'Donna Summer']['text']

lyrics_concat_donna = u''
for i in lyrics_data_donna.index:
    lyrics_concat_donna += lyrics_data_donna[i]
    
print("input chars length")
print(len(lyrics_concat_donna))
text = lyrics_concat_donna
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

input chars length
236908
corpus length: 236908
total chars: 73
nb sequences: 78956
Vectorization...
Build model...

--------------------------------------------------
Iteration 1
Epoch 1/1
78956/78956 [==============================] - 84s - loss: 2.2034    

----- diversity: 0.2
----- Generating with seed: "want to travel  
want to travel 'cross t"
want to travel  
want to travel 'cross to be  
  
The whe love  
I want a ray  
I want the me to so so the way  
I want to be the way  
I want to the a want to the way  
I want to be the way  
  
  
  
I want the way  
I way to so do  
The way heart  
I want to so the way  
  
  
  
  
I want to the way  
I want the way  
I want to me  
The way the way  
I want the way  
I want to so hear  
I want to the love  
I want the may  
  
  
  

----- diversity: 0.5
----- Generating with seed: "want to travel  
want to travel 'cross t"
want to travel  
want to travel 'cross to me  
I wart the the go, the wion  
I go  
  
He's the me me a day  
He'

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: divide by zero encountered in log


p night not a

----- diversity: 1.2
----- Generating with seed: "ou, love you, love you  
Love you, love "
ou, love you, love you  
Love you, love is the night  
Give your lows tryy  
Oh iten any uponeusasnond  
Swide wisino never dop s]es O  
enough that wasp toking never  ugomady  
Wel in retherinebet ees  
Of eys one  
he making , long,  
Ye it totrieeshe tuge onethay , onsing pcidead you, old of sois leg  
Andey he had that craght  
Dedpcufe up upincies  
The's to no  
I need  
S Tweevine wime  
couve, runohl  
Whicks 'cauethin',, my emt

--------------------------------------------------
Iteration 4
Epoch 1/1
78956/78956 [==============================] - 83s - loss: 2.1993    

----- diversity: 0.2
----- Generating with seed: "ordinary  
Modern day romance  
Uhhh yea"
ordinary  
Modern day romance  
Uhhh yeat the tithe fou to the  
I't to the  
I't to the the  
I't to to to to to to me to the me so to  
I't me  
I to the fee to to the feat to the me to the to to to to to to to to

If all ig the till okd sis so sen the sche to burce myse, ohooke lance so bter agade, aife  
enkesprake pracce comm what he plave  
I core neiliog

--------------------------------------------------
Iteration 11
Epoch 1/1
78956/78956 [==============================] - 77s - loss: 2.0239    

----- diversity: 0.2
----- Generating with seed: "o tragedy  
You've been gone much too lo"
o tragedy  
You've been gone much too long  
  
I moder  
I mads  
  
  
I mads  
  
I mads  
  
I mads  
  
  
I mads  
I mads  
  
I mads  
  
I'm my mody  
I mad  
  
I want to mad  
I man was  
  
I man me  
I'm my mad  
I moder  
I mads  
I man me  
I mads  
  
I don't was  
I mody  
  
I mads  
I'm my mads  
  
I mad  
I'm my mads  
I dair  
  
  
  
I mads  
I'm my mads  
  
  
I feel my mads  
I mads  
I mads  
  
  
  
I want t

----- diversity: 0.5
----- Generating with seed: "o tragedy  
You've been gone much too lo"
o tragedy  
You've been gone much too long  
I can't me  
  
I mads word  
It's o

(forever and ever we'll stay)  
  
  
  
  
  
I love  
  
  
I don't mass the lone  
  
I want the wore  
  
I want to man  
  
I want the way  
  
  
I'm the start  
  
  
I was the do  
I want the woll  
  
I want  
  
  
I love  
  
  
I want the to the lone  
  
  
  
I was  
I want the wore  
  
  
  
I'm lone  
  
I lone  
  
  
  
I love  
  
It all the lone  
  
  
It my the world  
I want the start  
  
I want the st

----- diversity: 0.5
----- Generating with seed: "you are  
(forever and ever we'll stay) "
you are  
(forever and ever we'll stay)  
  
  
  
I'm alone one at when the tale  
I see  
I don't be the stand we me  
  
I'm you'll through to wornating you're the looking  
I'm hyet that the to the way  
  
  
  
I want to to the lone  
I was to tream  
  
I want the do set won't masting  
I want that do the do  
  
  
  
I won't feel  
  
I don't don't me this stand to lone  
I'm on the hopen the way  
I won't stones  
I want you  

----- diversity: 1.0
----- Generat

...

  
I wanna could where your klope  
I wall the going all me  
I want all the well  
I can I wall the  
  
I goul  
I cough  
  
I kever love  
I want  
I could goup  
I cart  
I the love  
I wall  
  
It's feel love a unch  
  
I knews  
I could was  
I going  

----- diversity: 1.0
----- Generating with seed: "n inside  
A voice cried saying true lov"
n inside  
A voice cried saying true love  
  
I way  
I wanna feeer  
I can of the twect  
  
I guy  
  
I want  
  
I feel unuspesy to be  
I wanna  
I get y sliking he'p io, stop love  
  
I day you love take if leave  
I clound my gever  
I call someching  
I wagh never this guy nud on the living come  
  

I'll that ics forfveamed made to be get castatmery fadood  
I we a right  
  
I knews  
I coy noke  
  
  
leem thigh  
  
I ge

----- diversity: 1.2
----- Generating with seed: "n inside  
A voice cried saying true lov"
n inside  
A voice cried saying true love  
I feel this love  
Ilele and al lister  
I k've he made a out  
  


78956/78956 [==============================] - 90s - loss: 1.9578    

----- diversity: 0.2
----- Generating with seed: "  
  
My my Romeo, my my Romeo, my my Ro"
  
  
My my Romeo, my my Romeo, my my Romeos  
  
I love you  
  
  
  
  
I know the con  
  
I love you  
  
  
  
  
I love you  
  
  
I can  
  
  
I love you  
  
  
  
  
I can  
  
I was to see  
  
  
  
I know  
I say  
  
I love you  
  
  
  
  
  
I don't let in the see  
  
I can  
I believes  
  
  
I can  
  
  
  
  
I was  
  
  
  
I believes  
  
I know the see  
  
I love you  
  
  
  
  
  
  
  
  
I was  
  
  


----- diversity: 0.5
----- Generating with seed: "  
  
My my Romeo, my my Romeo, my my Ro"
  
  
My my Romeo, my my Romeo, my my Round  
I know on in the key  
  
I love you  
I kever  
  
  
I love to stared  
I belest  
  
  
I time  
  
  
  
  
  
  
  
  
  
I believed  
  
I believes on shiyed  
  
I cark  
I time  
I see  
I feel  
I've  
  
  
I can  
I need you  
  
I seive  
  
  


I want to see  
I want to me  
I can  
I want you  
I want it  
  
I want to live  
I want you would where it's a way  
I want it  
I can  
I want to love  
I want to want to was world  
I want to listen 

----- diversity: 0.5
----- Generating with seed: "
  
I want to sing hymns with Mahalia  
"

  
I want to sing hymns with Mahalia  
I seems  
  
  
I want you to affooh  
I want to love at song is all the wanes wat when he's a dould down  
It's a sidimed and when shems what she're all the sands  
I want sometheridws where you wonder  
I comy love  
I want our we heart  
I want to the wainnow in I love you won't love  
I can  
I want to see  
I'm wind in the wains the walls  
I made  
I was nightsise is is you down  
I love me 

----- diversity: 1.0
----- Generating with seed: "
  
I want to sing hymns with Mahalia  
"

  
I want to sing hymns with Mahalia  
I don't be what therend sunsing biadbest  
In mon  
If gut you worldnels when we losted out time out  
I'm wire owunny the sunll

Oh i
vitmmleteee otte sge n  ' bt 
hadey 
eoba dhvvvgyho 
theukis 
 ois,om  toa ,o' eoaren'rt  snyramidegetoetovg 
 cI Ire'  mode lohorgnieneee bdtomre 
hi n e hyomo   Ine'kopeortamn  ve as,sm sI ta gireaant u wce  
 ,hvdctInrioopgn s i  agm  do 'mhovocev esptg iu',  mvhe hemopllie ailoleldware yleci rve tabvuyre  a  moul mut ohore  we kene  oogidggv vokvdtecg tothne vo giec Iretvem  , bi r se a ksoo 

--------------------------------------------------
Iteration 47
Epoch 1/1
78956/78956 [==============================] - 87s - loss: 3.1188    

----- diversity: 0.2
----- Generating with seed: "tayed awake last night  
Told myself tha"
tayed awake last night  
Told myself thang  

  
oe  

  

  
he  
e  
e  
he  
e  
I  
eer  
o  
os  
e sheee whe  
our  
eethe  
e  

  
oe  
  

 wh  
oe  
  
he  
e 

  
e an  oheer  anehee wo  

w  
he  
e  

eee  
oe  
he  

  
he  
ene  
h so  
he  

  
o  
e  
he  
he  
ou the th so  
ous  
he n  

s  
o se  
heny  
h  
e  

 anee  

ee  
I yee sh

78956/78956 [==============================] - 88s - loss: 3.1596    

----- diversity: 0.2
----- Generating with seed: "ying that he'd give me a chance  
Hoping"
ying that he'd give me a chance  
Hoping  
of  

  
o  
o  
  
o  
  
or  
or  oe  
o   
o  
o  
  

 
o  

  
or  
on  
oo  
o  
o  
on  
oo  
o  

 
o  
o  
on  
  
o  
oo  
o  
of  
o  

  
o  
on  
or  
or  
on  
  

  
o  

os  
 
o  
  
oo   
 

  

  
o  

o   
oo  
o 
oo  

  
or  
o  
o  
 

  
ore  
  

  
o  
o  

o  
  
om  
of  

 
oo  

  
on  
or  
  
oo  
o  
o  
o  

 

o  
  

 
o 
o  
o  

  
o   
or  
o  
  
o  
o  


----- diversity: 0.5
----- Generating with seed: "ying that he'd give me a chance  
Hoping"
ying that he'd give me a chance  
Hoping mog  
y  
o  
r  
od no oh  
 to  
oot  
or  
o  
ar cr  
 io nh wo  

  
o hon  

  
or na  
oein  
  
hrr   
o ro t ye cor moo  
o toy  

  
t tore  

or te t wive for oh  
  
 no an tor ron  
oor  
on  
h y ire  
ohg   
h  
ohe  
o gn wo  
on  
ow  
h s lo ch

In [ ]:
# sentence 1

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = 'Loneliness is always looking for a frien'
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [ ]:
# sentence 2

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = "We go hard or we go home \nWe gon' do thi"
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [ ]:
# sentence 3

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = "At first I was afriad, I was petrified \n"
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [ ]:
# sentence 4

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = "Young Money, Nicki Minaj, Justin  \n  \nSh"
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [ ]:
# sentence 5

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = "Life is to be lived on the positive tip "
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [ ]:
# sentence 6

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = "Another night I'm waiting for you  \nWait"
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [ ]:
# sentence 7

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = 'Back track, listen, stop that  \nBefore e'
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [ ]:
# sentence 8

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = 'How can a non-musician discuss the futur'
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [ ]:
# sentence 9

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = 'Hi out there. This is Shaun Goldberg and'
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [ ]:
# sentence 10

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = "Whose eyes in my behind  \nI don't recogn"
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()